### Semantic kernel Vector Search with chat context/history 

In previous session, we walked through a simple example based on company ABC data. In that example, and employee wanted to simply connect to company's data, retrieve some info, and add a new record.

Now let's consider a different scenario. 

Let's assume the company has a chatbot that provides below basic support to customers:
1. Provide state of shipment
2. Provide state of return
2. Asking about product info and availability
3. Recommend best product based on customer needs

For an optimal experience, the chatbot needs to not only access to all product, sales, shipment and return data, but also it would need to be able to pick up the conversation with each customer where it was left off last

For this usecase, we are going to introduce and use below semantic kernel capabilities (in addition to what we covered before) to develop a solution:
- Chat completion **Agent** to manage the conversation and data access (one agent)
  -- example: we don't want the agent to access data for other customers or sensitive company data. 
- Vector search using Semantic Kernel Vector Store connectors
  -- Use company data as context to guide the conversation aka **RAG**
- Store and maintain **chat history** per customer and retrieve the history when needed.
  -- We will persist the history in CosmosDB



##### We start with importing the necessary libraries

In [1]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
# importing all we need for this session
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents import AgentRegistry, ChatHistoryAgentThread


### Expanding the plugin

In previous session, we defined a class that enabled some basic functionalities to interact with the database. Now let's think about what data would the chatbot need to access and create a plugin to pull in that data:
- Chatbot will start by asking the customer to provide their id as an identifier. Based on this id, it needs to access:
    - All sales data for the customer
    - All Return data for the customer
    - All basic customer info
    - All reviews and ratings provided by that customer
- Chatbot will also need to access data about all product offerings.

In [2]:
import psycopg2
from pandas import DataFrame
from typing import Optional
from semantic_kernel.functions import kernel_function


class ABC_ChatPlugin:
    def __init__(self, db_uri: str):
        self.conn = psycopg2.connect(db_uri)
        self.cursor = self.conn.cursor()
        print("Connected to company's database successfully.")
    
    @kernel_function
    async def get_all_products(self) -> list[dict]:
        """Gets all products info from the database."""
        query = """SELECT * FROM products;"""
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products = DataFrame(rows, columns=columns)
            return products.to_dict(orient="records")  # <-- JSON serializable
        except Exception as e:
            print(f"Error fetching products: {e}")
            return None

    @kernel_function
    async def get_product_info(self, product_name: Optional[str] = None, product_id: Optional[int] = None) -> list[dict]:
        """Retrieves product information based on product name or ID."""
        query = """SELECT 
                    product_id,                   
                    name,
                    inventory,
                    price,
                    refurbished,
                    category
                FROM products
                WHERE (LOWER(name) = LOWER(%(product_name)s) AND %(product_name)s IS NOT NULL)
                   OR (product_id = %(product_id)s AND %(product_id)s IS NOT NULL)
                   """
        if not product_name and not product_id:
            print("No valid product name or ID provided.")
            return None
        elif product_id:
            self.cursor.execute(query, {"product_name": None, "product_id": product_id})
        else:
            self.cursor.execute(query, {"product_name": product_name, "product_id": None})

            
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products= DataFrame(rows, columns=columns)
            products.to_dict(orient="records")  # <-- JSON serializabl
            
            return products.to_dict(orient="records")  # <-- JSON serializabl
        except Exception as e:
            print(f"Error fetching product information: {e}")
            return None
    
    async def get_customer_info(self, customer_email: str) -> list[dict]:
        """Gets all product information from the database."""
        query = """SELECT
                        customer_id,
                        city,
                        state,
                        country,
                        sentiment_score,
                        name,
                        email 
                    
                    FROM customers
                WHERE LOWER(email) = LOWER(%(customer_email)s);
                   """
        self.cursor.execute(query, {"customer_email": customer_email})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No record was found for customer with provided email: {customer_email}")
            return None
        
    @kernel_function
    def get_customer_sales_info(self, customer_email: str) -> Optional[dict]:
        """Returns all sales info for a customer"""
        query = """
            SELECT sales.sales_id, sales.customer_id, sales.quantity,
                   sales.product_id, sales.sale_date, products.name, products.price
            FROM sales
            JOIN products ON sales.customer_id = customers.customer_id
            WHERE LOWER(customers.email) = LOWER(%(customer_email)s);
        """
        self.cursor.execute(query, {"customer_email": customer_email})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No sales data found for customer with email: {customer_email}")
            return None
        
    @kernel_function
    def get_customer_returns_info(self, customer_email: str) -> Optional[dict]:
        """Returns all returns info for a customer"""
        query = """SELECT 
                        return_items.return_id,
                        return_items.sales_id,
                        return_items.return_status,
                        return_items.reason,
                        return_items.status_date,
                        sales.customer_id,
                        sales.product_id,
                        sales.sale_date
                    FROM 
                        return_items
                    JOIN 
                        sales ON return_items.sales_id = sales.sales_id
                    JOIN
                        customers ON sales.customer_id = customers.customer_id
                    WHERE 
                        LOWER(customers.email) = LOWER(%(customer_email)s);

        """
        self.cursor.execute(query, {"customer_email": customer_email})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No return data found for customer with email: {customer_email}")
            return None

    def close_connection(self):
        """Closes the database connection."""
        self.cursor.close()
        self.conn.close()
        print("Database connection closed.")
    



In [12]:
from src.get_conn import get_connection_uri

conn_uri = get_connection_uri()
abc_chat_kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()
abc_chat_kernel.add_service(chat_completion)


abc_chat_plugin = ABC_ChatPlugin(db_uri=conn_uri)

abc_chat_kernel.add_plugin(
    plugin = abc_chat_plugin,
    plugin_name = "ABC_Chat_Plugin"
) 

# # Enable planning
# execution_settings = AzureChatPromptExecutionSettings()
# execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# # Create a history of the conversation
# history = ChatHistory()


Connection uri was rertieved successfully.
Connected to company's database successfully.


KernelPlugin(name='ABC_Chat_Plugin', description=None, functions={'get_all_products': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='get_all_products', plugin_name='ABC_Chat_Plugin', description='Gets all products info from the database.', parameters=[], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='list[dict]', is_required=True, type_object=<class 'list'>, schema_data={'type': 'array'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x0000014BDB9B94D0>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x0000014BDB9B9790>, method=<bound method ABC_ChatPlugin.get_all_products of <__main__.ABC_ChatPlugin object at 0x0000014BDB1B7810>>, stream_method=None), 'get_customer_returns_info': KernelFunctionFromMethod(metadata=K

In [41]:
from dotenv import load_dotenv

from azure.cosmos import CosmosClient
from azure.identity import DefaultAzureCredential

import json
import os

load_dotenv()


client = CosmosClient.from_connection_string(os.getenv("COSMOS_CONNECTION_STRING"))

databaseName = "ABC"
database = client.get_database_client(databaseName)


containerName =  "customer_chats"
container = database.get_container_client(containerName)

In [92]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
from dataclasses import dataclass
from typing import Annotated
from semantic_kernel.agents import ChatHistoryAgentThread



from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents import ChatHistory, ChatMessageContent

class ChatHistoryInCosmosDB(ChatHistoryAgentThread):
    """This class stores the chat history in a Cosmos DB container as plain documents."""

    def __init__(self, session_id: str, user_id: str, container):
        super().__init__()
        self.session_id = session_id
        self.user_id = user_id
        self.container = container

    async def store_messages(self):
        """Store the chat history in the Cosmos DB as a document."""
        messages = [msg async for msg in self.get_messages()]
        item = {
            "id": self.session_id,
            "email": self.user_id,
            "messages": [msg.model_dump() for msg in messages],
        }
        try:
            res=self.container.upsert_item(item)
            print(f"Messages stored in Cosmos DB with response: {res}")
        except Exception as e:
            print(f"Error storing messages in Cosmos DB: {e}")
            raise

    async def read_messages(self):
        """Read the chat history from the Cosmos DB."""
        try:
            item = self.container.read_item(item=self.session_id, partition_key=self.user_id)
            print(f"Messages read from Cosmos DB")
            for m in item.get("messages", []):    
                # Convert each message to ChatMessageContent
                await self.on_new_message(ChatMessageContent.model_validate(m))
            return True
        except Exception as e:
            print(f"Error reading messages from Cosmos DB: {e}")
            return False

In [89]:
support_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="SupportAgent",
    kernel=abc_chat_kernel,
    instructions="You are a support agent for ABC Company. You can answer questions about products, customers, sales, and returns. Use the ABC_Chat_Plugin to access the company's database."
)

hist = ChatHistoryInCosmosDB(
    session_id="session1+jd@jd.ca",
    user_id="jd@jd.ca",
    container=container
)

In [53]:
response = await support_agent.get_response(messages="What products do we have?",thread=hist)
print(f"{response.message.name}: {response.message.content}")
await hist.store_messages()

SupportAgent: Here are the products we currently have at ABC Company:

Laptops:
- Macbook Pro M4 (High performance, for designers and gamers)
- Macbook Pro M3 (Refurbished)

Smartphones:
- iPhone 16 Pro (Latest model, cutting edge)
- iPhone 14 Pro (Latest features)
- iPhone 14 Pro Max (Refurbished)
- iPhone 14 (Refurbished)
- iPhone 13 (Great value)
- iPhone 12 (Refurbished)
- iPhone SE (Compact and budget-friendly)

Tablets:
- Tablet 0.2 (Refurbished)
- Tablet 0.2 new

Headphones:
- Jabra Headphone
- Jabra Headphone Elite
- Jabra Headphone Sport
- Jabra Headphone Studio
- Jabra Headphone Pro

TVs:
- Sony TV 55 inch
- Sony TV 65 inch (Refurbished)
- Sony TV 75 inch
- Sony TV OLED

If you need details about any product, let me know!
Messages stored in Cosmos DB with response: {'id': 'session1+jd@jd.ca', 'email': 'jd@jd.ca', 'messages': [{'ai_model_id': None, 'metadata': {}, 'content_type': 'message', 'role': 'user', 'name': None, 'items': [{'ai_model_id': None, 'metadata': {}, 'content_

In [93]:
hist2 = ChatHistoryInCosmosDB(
    session_id="session1+jd@jd.ca",
    user_id="jd@jd.ca",
    container=container
)


In [94]:
res = await hist2.read_messages()

Messages read from Cosmos DB


In [96]:
async for msg in hist2.get_messages():
    print(msg)

What products do we have?


Here are the products we currently have at ABC Company:

Laptops:
- Macbook Pro M4 (High performance, for designers and gamers)
- Macbook Pro M3 (Refurbished)

Smartphones:
- iPhone 16 Pro (Latest model, cutting edge)
- iPhone 14 Pro (Latest features)
- iPhone 14 Pro Max (Refurbished)
- iPhone 14 (Refurbished)
- iPhone 13 (Great value)
- iPhone 12 (Refurbished)
- iPhone SE (Compact and budget-friendly)

Tablets:
- Tablet 0.2 (Refurbished)
- Tablet 0.2 new

Headphones:
- Jabra Headphone
- Jabra Headphone Elite
- Jabra Headphone Sport
- Jabra Headphone Studio
- Jabra Headphone Pro

TVs:
- Sony TV 55 inch
- Sony TV 65 inch (Refurbished)
- Sony TV 75 inch
- Sony TV OLED

If you need details about any product, let me know!
